In [1]:
'''بِسْمِ اللهِ الرَّحْمٰنِ الرَّحِيْمِ'''

'بِسْمِ اللهِ الرَّحْمٰنِ الرَّحِيْمِ'

In [2]:
'''سُبْحَـٰنَكَ لَا عِلْمَ لَنَآ إِلَّا مَا عَلَّمْتَنَآ ۖ إِنَّكَ أَنتَ الْعَلِيمُ الْحَكِيمُ'''

'سُبْحَـٰنَكَ لَا عِلْمَ لَنَآ إِلَّا مَا عَلَّمْتَنَآ ۖ إِنَّكَ أَنتَ الْعَلِيمُ الْحَكِيمُ'

In [101]:
from IPython.core.display import HTML
HTML(r"""
<style>
    * {
        font-family: Monaco;
        # font-size: 21px !important;
    }
</style>
""")

In [43]:
%xmode Minimal
import os,sys,warnings,time,re,gc
from IPython.display import clear_output
warnings.filterwarnings("ignore")
from typing import Literal,Optional
from pathlib import Path
import numpy as np
import polars as pl
import polars.selectors as cs
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.templates["mod"] = go.layout.Template(layout=dict(font=dict(family="Monaco",size=12)))
pio.templates.default = "plotly_dark+mod"
from plotly.subplots import make_subplots
import hvplot as hv
os.cpu_count()

Exception reporting mode: Minimal


12

In [5]:
hv.extension('plotly')

In [6]:
pl.Config.set_fmt_float("full")
pl.Config.set_fmt_str_lengths(200)
pl.Config.set_float_precision(2)
pl.Config.set_tbl_rows(32)

polars.config.Config

In [7]:
path_to_train = Path("Downloads/credit_risk/train")
all_train_files = os.listdir(path_to_train)
print(len(all_train_files))
all_train_files.sort()
all_files_df = (
    pl.DataFrame({
        "index":range(len(all_train_files)),
        "filename":all_train_files
    },
    schema={"index":pl.Int8,"filename":pl.Utf8}
    )
)
all_files_df

32


index,filename
i8,str
0,"""train_applprev_1_0.parquet"""
1,"""train_applprev_1_1.parquet"""
2,"""train_applprev_2.parquet"""
3,"""train_base.parquet"""
4,"""train_credit_bureau_a_1_0.parquet"""
5,"""train_credit_bureau_a_1_1.parquet"""
6,"""train_credit_bureau_a_1_2.parquet"""
7,"""train_credit_bureau_a_1_3.parquet"""
8,"""train_credit_bureau_a_2_0.parquet"""


In [35]:
feature_columns = (
    pl.read_csv("Downloads/feature_definitions.csv")
    .select(
        pl.all().shrink_dtype()
    )
)

def rename_to_full(df:pl.LazyFrame) -> pl.DataFrame:    
    rename_dict = {}
    for _ in df.columns:
        des_str = feature_columns.filter(pl.col("Variable") == _)["Description"].item() if _ not in ["case_id","num_group1","num_group2"] else ""
        total_str = _+"\n"+"("+des_str+")" if _ not in ["case_id","num_group1","num_group2"] else _
        rename_dict[_] = total_str
    return df.rename(rename_dict)

def preprocess(ind:int) -> pl.LazyFrame:
    return (
        pl.scan_parquet(path_to_train/all_train_files[ind])
        .drop(cs.contains('date'))
        .select(pl.all().shrink_dtype())
        .pipe(rename_to_full)
    )

def plot_nulls(df:pl.LazyFrame):
    l = df.collect().shape[0]
    gc.collect()
    return (
        px.bar(
            df
            .null_count()
            .collect()
            .transpose(include_header=True)
            .with_columns(
                (pl.col("column_0")/l).round(2).alias("percent")
            ),
            x="column",
            y="column_0",
            color="column",
            hover_data=["column","percent"],
            hover_name="column",
            height=900
        )
        .update_xaxes(showticklabels=False,title="All Columns")
        .update_yaxes(title="Null Counts")
        .update_layout(legend_tracegroupgap=10)
        .add_hline(y=l,line=dict(dash="dashdot"),annotation_text="Total rows")
    )

print(gc.collect())
feature_columns.head()

1869


Variable,Description
str,str
"""actualdpd_943P""","""Days Past Due (DPD) of previous contract (actual)."""
"""actualdpdtolerance_344P""","""DPD of client with tolerance."""
"""addres_district_368M""","""District of the person's address."""
"""addres_role_871L""","""Role of person's address."""
"""addres_zip_823M""","""Zip code of the address."""


## Base Files

In [9]:
all_files_df.filter(pl.col("filename").str.contains("base"))

index,filename
i8,str
3,"""train_base.parquet"""


In [10]:
train_base = (
    pl.read_parquet(path_to_train/all_train_files[3])
    .select(pl.all().shrink_dtype())
    .with_columns(
        pl.col("date_decision").str.to_date(),
        pl.col("date_decision").str.to_date().dt.month().alias("month")
    )
)
gc.collect()
display(train_base.head())
train_base.estimated_size("mb")

case_id,date_decision,MONTH,WEEK_NUM,target,month
i32,date,i32,i8,i8,i8
0,2019-01-03,201901,0,0,1
1,2019-01-03,201901,0,0,1
2,2019-01-04,201901,0,0,1
3,2019-01-03,201901,0,0,1
4,2019-01-04,201901,0,1,1


21.83903217315674

In [11]:
train_base.describe()

statistic,case_id,date_decision,MONTH,WEEK_NUM,target,month
str,f64,str,f64,f64,f64,f64
"""count""",1526659.00,"""1526659""",1526659.00,1526659.00,1526659.00,1526659.00
"""null_count""",0.00,"""0""",0.00,0.00,0.00,0.00
"""mean""",1286076.57,"""2019-10-16""",201936.29,40.77,0.03,6.43
"""std""",718946.59,null,44.74,23.80,0.17,3.51
"""min""",0.00,"""2019-01-01""",201901.00,0.00,0.00,1.00
"""25%""",766198.00,"""2019-06-13""",201906.00,23.00,0.00,3.00
"""50%""",1357358.00,"""2019-10-11""",201910.00,40.00,0.00,7.00
"""75%""",1739023.00,"""2020-01-26""",202001.00,55.00,0.00,9.00
"""max""",2703454.00,"""2020-10-05""",202010.00,91.00,1.00,12.00


In [61]:
def missing_case_ids(df:pl.LazyFrame):
    return (
        df
        .with_context(
            train_base.lazy().select(pl.col("case_id").suffix("_base"))
        )
        .select(
            (~pl.col("case_id_base").is_in(pl.col("case_id"))).cast(pl.Int8).mean().alias("percentage of base case ids not present")
        )
        .collect()
    )


## Previous Files

In [99]:
all_files_df.filter(pl.col('filename').str.contains("prev"))

index,filename
i8,str
0,"""train_applprev_1_0.parquet"""
1,"""train_applprev_1_1.parquet"""
2,"""train_applprev_2.parquet"""


In [12]:
train_applprev_1 = preprocess(0)
train_applprev_2 = preprocess(1)
gc.collect()

0

In [44]:
display(train_applprev_1.fetch().head())
train_applprev_1.pipe(plot_nulls)

case_id,actualdpd_943P (Days Past Due (DPD) of previous contract (actual).),annuity_853A (Monthly annuity for previous applications.),byoccupationinc_3656910L (Applicant's income from previous applications.),cancelreason_3545846M (Application cancellation reason.),childnum_21L (Number of children in the previous application.),credacc_actualbalance_314A (Actual balance on credit account.),credacc_credlmt_575A (Credit card credit limit provided for previous applications.),credacc_maxhisbal_375A (Maximal historical balance of previous credit account),credacc_minhisbal_90A (Minimum historical balance of previous credit accounts.),credacc_status_367L (Account status of previous credit applications.),credacc_transactions_402L (Number of transactions made with the previous credit account of the applicant.),credamount_590A (Loan amount or card limit of previous applications.),credtype_587L (Credit type of previous application.),currdebt_94A (Previous application's current debt.),district_544M (District of the address used in the previous loan application.),downpmt_134A (Previous application downpayment amount.),dtlastpmt_581D (Date of last payment made by the applicant.),dtlastpmtallstes_3545839D (Date of the applicant's last payment.),education_1138M (Applicant's education level from their previous application.),employedfrom_700D (Employment start date from the previous application.),familystate_726L (Family State in previous application of applicant.),inittransactioncode_279L (Type of the initial transaction made in the previous application of the client.),isbidproduct_390L (Flag for determining if the product is a cross-sell in previous applications.),isdebitcard_527L (Previous application flag indicating if product being applied for is a debit card.),mainoccupationinc_437A (Client's main income amount in their previous application.),maxdpdtolerance_577P (Maximum DPD with tolerance (on previous application/s).),num_group1,outstandingdebt_522A (Amount of outstanding debt on the client's previous application.),pmtnum_8L (Number of payments made for the previous application.),postype_4733339M (Type of point of sale.),profession_152M (Profession of the client during their previous loan application.),rejectreason_755M (Reason for previous application rejection.),rejectreasonclient_4145042M (Reason for rejection of the client's previous application.),revolvingaccount_394A (Revolving account that was present in the applicant's previous application.),status_219L (Previous application status.),tenor_203L (Number of instalments in the previous application.)
i16,f32,f32,f32,str,f32,f32,f32,f32,f32,str,f32,f32,str,f32,str,f32,str,str,str,str,str,str,bool,bool,f32,f32,i8,f32,f32,str,str,str,str,f32,str,f32
2,0.00,640.20,null,"""a55475b1""",0.00,null,0.00,null,null,null,null,10000.00,"""CAL""",null,"""P136_108_173""",0.00,null,null,"""P97_36_170""","""2010-02-15""","""SINGLE""","""CASH""",false,null,8200.00,null,0,null,24.00,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,"""D""",24.00
2,0.00,1682.40,null,"""a55475b1""",0.00,null,0.00,null,null,null,null,16000.00,"""CAL""",null,"""P136_108_173""",0.00,null,null,"""P97_36_170""","""2010-02-15""","""SINGLE""","""CASH""",false,null,8200.00,null,1,null,12.00,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,"""D""",12.00
3,0.00,6140.00,null,"""P94_109_143""",null,null,0.00,null,null,null,null,59999.80,"""CAL""",null,"""P131_33_167""",0.00,null,null,"""P97_36_170""","""2018-05-15""","""MARRIED""","""CASH""",false,null,11000.00,null,0,null,12.00,"""a55475b1""","""a55475b1""","""P94_109_143""","""a55475b1""",null,"""D""",12.00
4,0.00,2556.60,null,"""P24_27_36""",null,null,0.00,null,null,null,null,40000.00,"""CAL""",null,"""P194_82_174""",0.00,null,null,"""a55475b1""",null,null,"""CASH""",false,null,16000.00,null,0,null,24.00,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,"""T""",24.00
5,0.00,null,null,"""P85_114_140""",null,null,null,null,null,null,null

In [42]:
display(train_applprev_2.fetch().head())
train_applprev_2.pipe(plot_nulls)

case_id,actualdpd_943P (Days Past Due (DPD) of previous contract (actual).),annuity_853A (Monthly annuity for previous applications.),byoccupationinc_3656910L (Applicant's income from previous applications.),cancelreason_3545846M (Application cancellation reason.),childnum_21L (Number of children in the previous application.),credacc_actualbalance_314A (Actual balance on credit account.),credacc_credlmt_575A (Credit card credit limit provided for previous applications.),credacc_maxhisbal_375A (Maximal historical balance of previous credit account),credacc_minhisbal_90A (Minimum historical balance of previous credit accounts.),credacc_status_367L (Account status of previous credit applications.),credacc_transactions_402L (Number of transactions made with the previous credit account of the applicant.),credamount_590A (Loan amount or card limit of previous applications.),credtype_587L (Credit type of previous application.),currdebt_94A (Previous application's current debt.),district_544M (District of the address used in the previous loan application.),downpmt_134A (Previous application downpayment amount.),dtlastpmt_581D (Date of last payment made by the applicant.),dtlastpmtallstes_3545839D (Date of the applicant's last payment.),education_1138M (Applicant's education level from their previous application.),employedfrom_700D (Employment start date from the previous application.),familystate_726L (Family State in previous application of applicant.),inittransactioncode_279L (Type of the initial transaction made in the previous application of the client.),isbidproduct_390L (Flag for determining if the product is a cross-sell in previous applications.),isdebitcard_527L (Previous application flag indicating if product being applied for is a debit card.),mainoccupationinc_437A (Client's main income amount in their previous application.),maxdpdtolerance_577P (Maximum DPD with tolerance (on previous application/s).),num_group1,outstandingdebt_522A (Amount of outstanding debt on the client's previous application.),pmtnum_8L (Number of payments made for the previous application.),postype_4733339M (Type of point of sale.),profession_152M (Profession of the client during their previous loan application.),rejectreason_755M (Reason for previous application rejection.),rejectreasonclient_4145042M (Reason for rejection of the client's previous application.),revolvingaccount_394A (Revolving account that was present in the applicant's previous application.),status_219L (Previous application status.),tenor_203L (Number of instalments in the previous application.)
i32,f32,f32,f32,str,f32,f32,f32,f32,f32,str,f32,f32,str,f32,str,f32,str,str,str,str,str,str,bool,bool,f32,f32,i8,f32,f32,str,str,str,str,f32,str,f32
40704,0.00,7204.60,null,"""P94_109_143""",null,null,0.00,null,null,null,null,54000.00,"""CAL""",null,"""P147_6_101""",0.00,null,null,"""a55475b1""",null,null,"""CASH""",false,null,40000.00,null,0,null,12.00,"""P46_145_78""","""a55475b1""","""P198_131_9""","""P94_109_143""",null,"""D""",12.00
40734,0.00,3870.20,null,"""P94_109_143""",null,null,0.00,null,null,null,null,50000.00,"""CAL""",null,"""P111_148_100""",0.00,null,null,"""a55475b1""",null,null,"""CASH""",false,null,50000.00,null,0,null,18.00,"""P149_40_170""","""a55475b1""","""P45_84_106""","""P94_109_143""",null,"""D""",18.00
40737,0.00,2324.40,1.00,"""a55475b1""",0.00,null,0.00,null,null,null,null,30000.00,"""CAL""",0.00,"""a55475b1""",0.00,null,null,"""P97_36_170""","""2014-01-15""","""MARRIED""","""CASH""",false,null,16000.00,null,0,0.00,18.00,"""P46_145_78""","""a55475b1""","""a55475b1""","""a55475b1""",null,"""D""",18.00
40791,0.00,3044.40,null,"""P94_109_143""",null,null,0.00,null,null,null,null,44840.00,"""COL""",null,"""P19_11_176""",0.00,null,null,"""a55475b1""",null,null,"""POS""",false,null,30000.00,null,0,null,24.00,"""P177_117_192""","""a55475b1""","""P99_56_166""","""P94_109_143""",null,"""D""",24.00
40791,0.00,2320.80,1.00,"""a55475b1""",0.00,null,0.00,null,null,null,null

In [70]:
train_previous_application = pl.concat(
    [
        train_applprev_1.with_columns(pl.col("case_id").cast(pl.Int32)),
        train_applprev_2.with_columns(pl.col("case_id").cast(pl.Int32))
    ],
    how="vertical",
    rechunk=True
)
train_previous_application.fetch().head()

case_id,actualdpd_943P (Days Past Due (DPD) of previous contract (actual).),annuity_853A (Monthly annuity for previous applications.),byoccupationinc_3656910L (Applicant's income from previous applications.),cancelreason_3545846M (Application cancellation reason.),childnum_21L (Number of children in the previous application.),credacc_actualbalance_314A (Actual balance on credit account.),credacc_credlmt_575A (Credit card credit limit provided for previous applications.),credacc_maxhisbal_375A (Maximal historical balance of previous credit account),credacc_minhisbal_90A (Minimum historical balance of previous credit accounts.),credacc_status_367L (Account status of previous credit applications.),credacc_transactions_402L (Number of transactions made with the previous credit account of the applicant.),credamount_590A (Loan amount or card limit of previous applications.),credtype_587L (Credit type of previous application.),currdebt_94A (Previous application's current debt.),district_544M (District of the address used in the previous loan application.),downpmt_134A (Previous application downpayment amount.),dtlastpmt_581D (Date of last payment made by the applicant.),dtlastpmtallstes_3545839D (Date of the applicant's last payment.),education_1138M (Applicant's education level from their previous application.),employedfrom_700D (Employment start date from the previous application.),familystate_726L (Family State in previous application of applicant.),inittransactioncode_279L (Type of the initial transaction made in the previous application of the client.),isbidproduct_390L (Flag for determining if the product is a cross-sell in previous applications.),isdebitcard_527L (Previous application flag indicating if product being applied for is a debit card.),mainoccupationinc_437A (Client's main income amount in their previous application.),maxdpdtolerance_577P (Maximum DPD with tolerance (on previous application/s).),num_group1,outstandingdebt_522A (Amount of outstanding debt on the client's previous application.),pmtnum_8L (Number of payments made for the previous application.),postype_4733339M (Type of point of sale.),profession_152M (Profession of the client during their previous loan application.),rejectreason_755M (Reason for previous application rejection.),rejectreasonclient_4145042M (Reason for rejection of the client's previous application.),revolvingaccount_394A (Revolving account that was present in the applicant's previous application.),status_219L (Previous application status.),tenor_203L (Number of instalments in the previous application.)
i32,f32,f32,f32,str,f32,f32,f32,f32,f32,str,f32,f32,str,f32,str,f32,str,str,str,str,str,str,bool,bool,f32,f32,i8,f32,f32,str,str,str,str,f32,str,f32
2,0.00,640.20,null,"""a55475b1""",0.00,null,0.00,null,null,null,null,10000.00,"""CAL""",null,"""P136_108_173""",0.00,null,null,"""P97_36_170""","""2010-02-15""","""SINGLE""","""CASH""",false,null,8200.00,null,0,null,24.00,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,"""D""",24.00
2,0.00,1682.40,null,"""a55475b1""",0.00,null,0.00,null,null,null,null,16000.00,"""CAL""",null,"""P136_108_173""",0.00,null,null,"""P97_36_170""","""2010-02-15""","""SINGLE""","""CASH""",false,null,8200.00,null,1,null,12.00,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,"""D""",12.00
3,0.00,6140.00,null,"""P94_109_143""",null,null,0.00,null,null,null,null,59999.80,"""CAL""",null,"""P131_33_167""",0.00,null,null,"""P97_36_170""","""2018-05-15""","""MARRIED""","""CASH""",false,null,11000.00,null,0,null,12.00,"""a55475b1""","""a55475b1""","""P94_109_143""","""a55475b1""",null,"""D""",12.00
4,0.00,2556.60,null,"""P24_27_36""",null,null,0.00,null,null,null,null,40000.00,"""CAL""",null,"""P194_82_174""",0.00,null,null,"""a55475b1""",null,null,"""CASH""",false,null,16000.00,null,0,null,24.00,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,"""T""",24.00
5,0.00,null,null,"""P85_114_140""",null,null,null,null,null,null,null

In [72]:
train_previous_application_status = preprocess(2)
train_previous_application_status.fetch().head()

case_id,cacccardblochreas_147M (Card blocking reason.),conts_type_509L (Person contact type in previous application.),credacc_cards_status_52L (Card status of the previous credit account.),num_group1,num_group2
i16,str,str,str,i8,i8
2,null,"""EMPLOYMENT_PHONE""",null,1,1
2,null,"""EMPLOYMENT_PHONE""",null,0,1
2,null,"""PRIMARY_MOBILE""",null,0,0
2,null,"""PRIMARY_MOBILE""",null,1,0
3,null,"""PRIMARY_MOBILE""",null,0,1


In [73]:
train_previous_application_status.pipe(plot_nulls)

## Static Files

In [74]:
all_files_df.filter(pl.col("filename").str.contains("static"))

index,filename
i8,str
26,"""train_static_0_0.parquet"""
27,"""train_static_0_1.parquet"""
28,"""train_static_cb_0.parquet"""


In [75]:
train_static_0 = preprocess(26)
print(train_static_0.collect().shape)
gc.collect()
train_static_0.fetch().head()

(1003757, 157)


case_id,actualdpdtolerance_344P (DPD of client with tolerance.),amtinstpaidbefduel24m_4187115A (Number of instalments paid before due date in the last 24 months.),annuity_780A (Monthly annuity amount.),annuitynextmonth_57A (Next month's amount of annuity.),applicationcnt_361L (Number of applications associated with the same email address as the client.),applications30d_658L (Number of applications made by the client in the last 30 days.),applicationscnt_1086L (Number of applications associated with the same phone number.),applicationscnt_464L (Number of applications made in the last 30 days by other clients with the same employer as the applicant.),applicationscnt_629L (Number of applications with the same employer in the last 7 days.),applicationscnt_867L (Number of applications associated with the same mobile phone.),avgdbddpdlast24m_3658932P (Average days past or before due of payment during the last 24 months.),avgdbddpdlast3m_4187120P (Average days past or before due of payment during the last 3 months.),avgdbdtollast24m_4525197P (Average days of payment before due date within the last 24 months (with tolerance).),"avgdpdtolclosure24_3658938P (Average DPD (days past due) with tolerance within the past 24 months from the maximum closure date, assuming that the contract is finished. If the contract is ongoing, the calculation is based on the current date.)",avginstallast24m_3658937A (Average instalments paid by the client over the past 24 months.),avglnamtstart24m_4525187A (Average loan amount in the last 24 months.),avgmaxdpdlast9m_3716943P (Average Days Past Due (DPD) of the client in last 9 months.),avgoutstandbalancel6m_4187114A (Average outstanding balance of applicant for the last 6 months.),avgpmtlast12m_4525200A (Average of payments made by the client in the last 12 months.),bankacctype_710L (Type of applicant's bank account.),cardtype_51L (Type of credit card.),clientscnt12m_3712952L (Number of clients that have used the same mobile phone as the applicant in the past 12 months.),clientscnt3m_3712950L (Number of clients who have the same mobile phone number in the last 3 months.),clientscnt6m_3712949L (Total number of clients who have used the same mobile number in the last 6 months.),clientscnt_100L (Number of applications with matching employer's phone and client's.),clientscnt_1022L (Number of clients sharing the same mobile phone.),clientscnt_1071L (Number of applications where the alternative phone number matches that of the client.),clientscnt_1130L (Number of applications where client's phone number matches the alternative phone contact.),clientscnt_136L (Number of applications associated with same email address as client's email.),clientscnt_157L (Number of clients whose employer has the same phone number as the client.),clientscnt_257L (Number of clients that share an alternative phone number with the applicant.),clientscnt_304L (Number of clients with the same phone number.),clientscnt_360L (Number of clients that have the same alternative phone number and employer's phone number.),clientscnt_493L (Number of clients with matching phone numbers for both the employer and the client.),clientscnt_533L (Number of clients with same client's and alternative's phone number),clientscnt_887L (Number of clients sharing the same employer's phone number.),…,numinstpaidearlyest_4493214L (Number of instalments paid before the due date.),numinstpaidlastcontr_4325080L (Number of paid installments from the client's last contract.),numinstpaidlate1d_3546852L (Number of instalments paid more than 1 day past their due date.),numinstregularpaid_973L (Number of fully paid regular installments in the client's previous contracts.),numinstregularpaidest_4493210L (Number of fully paid regular installments on clients' previous contracts.),numinsttopaygr_769L (Number of unpaid instalments.),numinsttopaygrest_4493213L (Number of unpaid instalments.),numinstunpaidmax_3546851L (Maximum number of unpaid instalments.),numinstunpaidmaxest_4

In [76]:
train_static_1 = preprocess(27)
print(train_static_1.collect().shape)
gc.collect()
train_static_1.fetch().head()

(522902, 157)


case_id,actualdpdtolerance_344P (DPD of client with tolerance.),amtinstpaidbefduel24m_4187115A (Number of instalments paid before due date in the last 24 months.),annuity_780A (Monthly annuity amount.),annuitynextmonth_57A (Next month's amount of annuity.),applicationcnt_361L (Number of applications associated with the same email address as the client.),applications30d_658L (Number of applications made by the client in the last 30 days.),applicationscnt_1086L (Number of applications associated with the same phone number.),applicationscnt_464L (Number of applications made in the last 30 days by other clients with the same employer as the applicant.),applicationscnt_629L (Number of applications with the same employer in the last 7 days.),applicationscnt_867L (Number of applications associated with the same mobile phone.),avgdbddpdlast24m_3658932P (Average days past or before due of payment during the last 24 months.),avgdbddpdlast3m_4187120P (Average days past or before due of payment during the last 3 months.),avgdbdtollast24m_4525197P (Average days of payment before due date within the last 24 months (with tolerance).),"avgdpdtolclosure24_3658938P (Average DPD (days past due) with tolerance within the past 24 months from the maximum closure date, assuming that the contract is finished. If the contract is ongoing, the calculation is based on the current date.)",avginstallast24m_3658937A (Average instalments paid by the client over the past 24 months.),avglnamtstart24m_4525187A (Average loan amount in the last 24 months.),avgmaxdpdlast9m_3716943P (Average Days Past Due (DPD) of the client in last 9 months.),avgoutstandbalancel6m_4187114A (Average outstanding balance of applicant for the last 6 months.),avgpmtlast12m_4525200A (Average of payments made by the client in the last 12 months.),bankacctype_710L (Type of applicant's bank account.),cardtype_51L (Type of credit card.),clientscnt12m_3712952L (Number of clients that have used the same mobile phone as the applicant in the past 12 months.),clientscnt3m_3712950L (Number of clients who have the same mobile phone number in the last 3 months.),clientscnt6m_3712949L (Total number of clients who have used the same mobile number in the last 6 months.),clientscnt_100L (Number of applications with matching employer's phone and client's.),clientscnt_1022L (Number of clients sharing the same mobile phone.),clientscnt_1071L (Number of applications where the alternative phone number matches that of the client.),clientscnt_1130L (Number of applications where client's phone number matches the alternative phone contact.),clientscnt_136L (Number of applications associated with same email address as client's email.),clientscnt_157L (Number of clients whose employer has the same phone number as the client.),clientscnt_257L (Number of clients that share an alternative phone number with the applicant.),clientscnt_304L (Number of clients with the same phone number.),clientscnt_360L (Number of clients that have the same alternative phone number and employer's phone number.),clientscnt_493L (Number of clients with matching phone numbers for both the employer and the client.),clientscnt_533L (Number of clients with same client's and alternative's phone number),clientscnt_887L (Number of clients sharing the same employer's phone number.),…,numinstpaidearlyest_4493214L (Number of instalments paid before the due date.),numinstpaidlastcontr_4325080L (Number of paid installments from the client's last contract.),numinstpaidlate1d_3546852L (Number of instalments paid more than 1 day past their due date.),numinstregularpaid_973L (Number of fully paid regular installments in the client's previous contracts.),numinstregularpaidest_4493210L (Number of fully paid regular installments on clients' previous contracts.),numinsttopaygr_769L (Number of unpaid instalments.),numinsttopaygrest_4493213L (Number of unpaid instalments.),numinstunpaidmax_3546851L (Maximum number of unpaid instalments.),numinstunpaidmaxest_4

In [77]:
train_static_2 = preprocess(28)
print(train_static_2.collect().shape)
gc.collect()
train_static_2.fetch().head()

(1500476, 44)


case_id,contractssum_5085716L (Total sum of values of contracts retrieved from external credit bureau.),days120_123L (Number of credit bureau queries for the last 120 days.),days180_256L (Number of credit bureau queries for last 180 days.),days30_165L (Number of credit bureau queries for the last 30 days.),days360_512L (Number of Credit Bureau queries for last 360 days.),days90_310L (Number of credit bureau queries for the last 90 days.),description_5085714M (Categorization of clients by credit bureau.),education_1103M (Level of education of the client provided by external source.),education_88M (Education level of the client.),firstquarter_103L (Number of results obtained from credit bureau in the first quarter.),for3years_128L (Number of rejected applications in the past 3 years.),for3years_504L (Client's credit history data over the last three years.),for3years_584L (Number of cancellations in the last 3 years.),formonth_118L (Number of rejections in a month.),formonth_206L (Number of cancelations in the previous month.),formonth_535L (Credit history for the last month.),forquarter_1017L (Number of cancellations recorded in the credit bureau in the last quarter.),forquarter_462L (Number of credit applications that were rejected in the last quarter.),forquarter_634L (Credit history for the last quarter.),fortoday_1092L (Client's credit history for today.),forweek_1077L (Number of cancelations in the last week.),forweek_528L (Credit history for the last week.),forweek_601L (Number of rejected applications in the last week.),foryear_618L (Number of application rejections in the previous year.),foryear_818L (Number of cancelations that occurred in last year.),foryear_850L (Credit history for the last year.),fourthquarter_440L (Number of results in fourth quarter.),maritalst_385M (Marital status of the client.),maritalst_893M (Marital status of the client),numberofqueries_373L (Number of queries to credit bureau.),pmtaverage_3A (Average of tax deductions.),pmtaverage_4527227A (Average of tax deductions.),pmtaverage_4955615A (Average of tax deductions.),pmtcount_4527229L (Number of tax deductions.),pmtcount_4955617L (Number of tax deductions.),pmtcount_693L (Number of tax deductions.),pmtscount_423L (Number of tax deduction payments.),pmtssum_45A (Sum of tax deductions for the client.),requesttype_4525192L (Tax authority request type.),riskassesment_302T (Estimated probability that the client will default on their credit obligation within the next year.),riskassesment_940T (Estimate of client's creditworthiness.),secondquarter_766L (Number of results in second quarter.),thirdquarter_1082L (Number of results in third quarter.)
i16,f32,f32,f32,f32,f32,f32,str,str,str,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,str,str,f32,f32,f32,f32,f32,f32,f32,f32,f32,str,str,f32,f32,f32
357,null,null,null,null,null,null,"""a55475b1""","""a55475b1""","""a55475b1""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""a55475b1""","""a55475b1""",null,null,null,null,null,null,null,6.00,6301.40,null,null,null,null,null
381,null,null,null,null,null,null,"""a55475b1""","""a55475b1""","""a55475b1""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""a55475b1""","""a55475b1""",null,null,null,null,null,null,null,6.00,4019.60,null,null,null,null,null
388,null,6.00,8.00,2.00,10.00,4.00,"""a55475b1""","""a55475b1""","""a55475b1""",2.00,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,6.00,"""a55475b1""","""a55475b1""",10.00,null,null,null,null,null,null,6.00,14548.00,null,null,null,3.00,5.00
405,null,0.00,0.00,0.00,1.00,0.00,"""a55475b1""","""a55475b1""","""a55475b1""",0.00,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,4.00,"""a55475b1""","""a55475b1""",1.00,null,null,null,null,null,null,6.00,10498.24,null,null,null,2.00,0.00
409,null,2.00,3.00,0.00,3.00,1.00,"""a55475b1""","""717ddd49"

## Train debitcard

In [97]:
all_files_df.filter(pl.col('filename').str.contains("debit"))

index,filename
i8,str
21,"""train_debitcard_1.parquet"""


In [78]:
train_debit = preprocess(21)
display(train_debit.fetch().head())
gc.collect()
train_debit.pipe(plot_nulls)

case_id,last180dayaveragebalance_704A (Average balance on debit card in the last 180 days.),last180dayturnover_1134A (Debit card's turnover within the last 180 days.),last30dayturnover_651A (Debit card turnover for the last 30 days.),num_group1
i16,f32,f32,f32,i8
225,null,null,null,0
331,null,null,null,0
358,null,null,null,0
390,null,null,null,0
390,null,null,null,2


In [98]:
all_files_df.filter(pl.col('filename').str.contains("deposit"))

index,filename
i8,str
22,"""train_deposit_1.parquet"""


In [79]:
train_deposit = preprocess(22)
display(train_deposit.fetch().head())
gc.collect()
train_deposit.pipe(plot_nulls)

case_id,amount_416A (Deposit amount.),num_group1
i16,f32,i8
225,0.00,0
331,260.37,0
358,0.00,0
390,211748.53,0
390,223.68,2


## Train person

In [95]:
all_files_df.filter(pl.col("filename").str.contains("person"))

index,filename
i8,str
24,"""train_person_1.parquet"""
25,"""train_person_2.parquet"""


In [91]:
train_person_1 = preprocess(24)
display(train_person_1.fetch().head())
gc.collect()
train_person_1.pipe(plot_nulls)

case_id,birth_259D (Date of birth of the person.),childnum_185L (Number of children of the applicant.),contaddr_district_15M (Zip code of a contact person's address.),contaddr_matchlist_1032L (Indicates whether the contact address is found in a code list.),contaddr_smempladdr_334L (Indicates whether the contact address is the same as the employment address.),contaddr_zipcode_807M (Zip code of contact address.),education_927M (Education level of the person.),empl_employedfrom_271D (Start date of employment.),empl_employedtotal_800L (Employment length of a person.),empl_industry_691L (Employment Industry of the person.),empladdr_district_926M (District where the employer's address is located.),empladdr_zipcode_114M (Zipcode of employer's address.),familystate_447L (Family state of the person.),gender_992L (Gender of a person.),housetype_905L (House type of the person.),housingtype_772L (Type of housing of the person.),incometype_1044T (Type of income of the person),isreference_387L (Flag indicating whether the person is a reference contact.),language1_981M (The primary language of the person.),mainoccupationinc_384A (Amount of the main income of the client.),maritalst_703L (Marital status of the client.),num_group1,personindex_1023L (Order of the person specified on the application form.),persontype_1072L (Person type.),persontype_792L (Person type.),registaddr_district_1083M (District of person's registered address.),registaddr_zipcode_184M (Registered address's zip code of a person.),relationshiptoclient_415T (Relationship to the client.),relationshiptoclient_642T (Relationship to the client.),remitter_829L (Flag indicating whether the client is a remitter.),role_1084L (Type of contact role.),role_993L (Person's role.),safeguarantyflag_411L (Flag indicating if client is using a flexible product with additional safeguard garanty.),sex_738L (Gender of the client.),type_25L (Contact type of a person.)
i16,str,f32,str,bool,bool,str,str,str,str,str,str,str,str,str,str,str,str,bool,str,f32,str,i8,f32,f32,f32,str,str,str,str,bool,str,str,bool,str,str
0,"""1986-07-01""",null,"""P88_18_84""",false,false,"""P167_100_165""","""P97_36_170""","""2017-09-15""","""MORE_FIVE""","""OTHER""","""P142_57_166""","""P167_100_165""","""MARRIED""",null,null,null,"""SALARIED_GOVT""",null,"""P10_39_147""",10800.00,null,0,0.00,1.00,1.00,"""P88_18_84""","""P167_100_165""",null,null,null,"""CL""",null,true,"""F""","""PRIMARY_MOBILE"""
0,null,null,"""a55475b1""",null,null,"""a55475b1""","""a55475b1""",null,null,null,"""a55475b1""","""a55475b1""",null,null,null,null,null,null,"""a55475b1""",null,null,1,1.00,1.00,4.00,"""a55475b1""","""a55475b1""","""SPOUSE""",null,false,"""EM""",null,null,null,"""PHONE"""
0,null,null,"""a55475b1""",null,null,"""a55475b1""","""a55475b1""",null,null,null,"""a55475b1""","""a55475b1""",null,null,null,null,null,null,"""a55475b1""",null,null,2,2.00,4.00,5.00,"""a55475b1""","""a55475b1""","""COLLEAGUE""","""SPOUSE""",false,"""PE""",null,null,null,"""PHONE"""
0,null,null,"""a55475b1""",null,null,"""a55475b1""","""a55475b1""",null,null,null,"""a55475b1""","""a55475b1""",null,null,null,null,null,null,"""a55475b1""",null,null,3,null,5.00,null,"""a55475b1""","""a55475b1""",null,"""COLLEAGUE""",null,"""PE""",null,null,null,"""PHONE"""
1,"""1957-08-01""",null,"""P103_93_94""",false,false,"""P176_37_166""","""P97_36_170""","""2008-10-29""","""MORE_FIVE""","""OTHER""","""P49_46_174""","""P160_59_140""","""DIVORCED""",null,null,null,"""SALARIED_GOVT""",null,"""P10_39_147""",10000.00,null,0,0.00,1.00,1.00,"""P103_93_94""","""P176_37_166""",null,null,null,"""CL""",null,true,"""M""","""PRIMARY_MOBILE"""


In [82]:
train_person_2 = preprocess(25)
display(train_person_2.fetch().head())
gc.collect()
train_person_2.pipe(plot_nulls)

case_id,addres_district_368M (District of the person's address.),addres_role_871L (Role of person's address.),addres_zip_823M (Zip code of the address.),conts_role_79M (Type of contact role of a person.),"empls_economicalst_849M (The economical status of the person (num_group1 - person, num_group2 - employment).)","empls_employedfrom_796D (Start of employment (num_group1 - person, num_group2 - employment).)","empls_employer_name_740M (Employer's name (num_group1 - person, num_group2 - employment).)",num_group1,num_group2,"relatedpersons_role_762T (Relationship type of a client's related person (num_group1 - person, num_group2 - related person).)"
i16,str,str,str,str,str,str,str,i8,i8,str
5,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""a55475b1""",null,"""a55475b1""",0,0,null
6,"""P55_110_32""","""CONTACT""","""P10_68_40""","""P38_92_157""","""P164_110_33""",null,"""a55475b1""",0,0,null
6,"""P55_110_32""","""PERMANENT""","""P10_68_40""","""a55475b1""","""a55475b1""",null,"""a55475b1""",0,1,null
6,"""P204_92_178""","""CONTACT""","""P65_136_169""","""P38_92_157""","""P164_110_33""",null,"""a55475b1""",1,0,"""OTHER_RELATIVE"""
6,"""P191_109_75""","""CONTACT""","""P10_68_40""","""P7_147_157""","""a55475b1""",null,"""a55475b1""",1,1,"""OTHER_RELATIVE"""


In [94]:
all_files_df.filter(pl.col('filename').str.contains("other"))

index,filename
i8,str
23,"""train_other_1.parquet"""


In [83]:
train_other = preprocess(23)
display(train_other.fetch().head())
train_other.pipe(plot_nulls)

case_id,amtdebitincoming_4809443A (Incoming debit card transactions amount.),amtdebitoutgoing_4809440A (Outgoing debit card transactions amount.),amtdepositbalance_4809441A (Deposit balance of client.),amtdepositincoming_4809444A (Amount of incoming deposits to client's account.),amtdepositoutgoing_4809442A (Amount of outgoing deposits from client's account.),num_group1
i32,f32,f32,f32,f32,f32,i8
43801,12466.60,12291.20,914.20,0.00,304.80,0
43991,3333.40,3273.40,0.00,0.00,0.00,0
44001,10000.00,10000.00,0.00,0.00,0.00,0
44053,0.00,0.00,2586.40,0.00,88.80,0
44130,63.80,60.80,0.00,0.00,0.00,0


## Train Tax Registry

In [93]:
all_files_df.filter(pl.col("filename").str.contains("registry"))

index,filename
i8,str
29,"""train_tax_registry_a_1.parquet"""
30,"""train_tax_registry_b_1.parquet"""
31,"""train_tax_registry_c_1.parquet"""


In [84]:
train_tax_registry_1 = preprocess(29)
display(train_tax_registry_1.fetch().head())
train_tax_registry_1.pipe(plot_nulls)

case_id,amount_4527230A (Tax deductions amount tracked by the government registry.),name_4527232M (Name of employer.),num_group1
i16,f32,str,i8
28631,1946.00,"""f980a1ea""",2
28631,711.00,"""f980a1ea""",3
28631,3616.40,"""f980a1ea""",0
28631,2600.00,"""f980a1ea""",1
28632,2564.00,"""5f9b74f5""",5


In [85]:
train_tax_registry_2 = preprocess(30)
display(train_tax_registry_2.fetch().head())
train_tax_registry_2.pipe(plot_nulls)

case_id,amount_4917619A (Tax deductions amount tracked by the government registry.),name_4917606M (Name of employer.),num_group1
i32,f32,str,i8
49435,6885.00,"""6b730375""",7
49435,6885.00,"""6b730375""",1
49435,6885.00,"""6b730375""",8
49435,6885.00,"""6b730375""",3
49435,6885.00,"""6b730375""",4


In [86]:
train_tax_registry_3 = preprocess(31)
display(train_tax_registry_3.fetch().head())
train_tax_registry_3.pipe(plot_nulls)

case_id,employername_160M (Employer's name.),num_group1,pmtamount_36A (Tax deductions amount for credit bureau payments.)
i16,str,i8,f32
357,"""c91b12ff""",5,1100.00
357,"""c91b12ff""",1,1200.00
357,"""c91b12ff""",4,1200.00
357,"""c91b12ff""",0,1200.00
357,"""c91b12ff""",3,628.60


## Train credit Bureau

In [87]:
(
    all_files_df
    .filter(~pl.col("filename").str.contains_any(["applprev","base","debit","other","person","static","registry","deposit"]))
)

index,filename
i8,str
4,"""train_credit_bureau_a_1_0.parquet"""
5,"""train_credit_bureau_a_1_1.parquet"""
6,"""train_credit_bureau_a_1_2.parquet"""
7,"""train_credit_bureau_a_1_3.parquet"""
8,"""train_credit_bureau_a_2_0.parquet"""
9,"""train_credit_bureau_a_2_1.parquet"""
10,"""train_credit_bureau_a_2_10.parquet"""
11,"""train_credit_bureau_a_2_2.parquet"""
12,"""train_credit_bureau_a_2_3.parquet"""


In [100]:
train_credit_1 = preprocess(4)
display(train_credit_1.fetch().head())
gc.collect()
train_credit_1.pipe(plot_nulls)

case_id,annualeffectiverate_199L (Interest rate of the closed contracts.),annualeffectiverate_63L (Interest rate for the active contracts.),classificationofcontr_13M (Classificiation of the active contract.),classificationofcontr_400M (Classificiation of the closed contract.),contractst_545M (Contract status.),contractst_964M (Contract status of terminated credit contract.),contractsum_5085717L (Sum of other contract values.),credlmt_230A (Credit limit of the closed credit contracts from credit bureau.),credlmt_935A (Credit limit for active loan.),debtoutstand_525A (Outstanding amount of existing contract.),debtoverdue_47A (Amount that is currently past due on a client's existing credit contract.),description_351M (Categorization of clients by credit bureau.),dpdmax_139P (Maximal days past due for active contract.),dpdmax_757P (Maximum days past due for a closed contract.),financialinstitution_382M (Name of financial institution that is linked to a closed contract.),financialinstitution_591M (Financial institution name of the active contract.),instlamount_768A (Instalment amount for the active contract in credit bureau.),instlamount_852A (Instalment amount for closed contract.),interestrate_508L (Interest rate for a closed contract in the credit bureau.),monthlyinstlamount_332A (Monthly instalment amount for active contract.),monthlyinstlamount_674A (Monthly amount of instalment payment on a closed contract.),nominalrate_281L (Interest rate of the active contract.),nominalrate_498L (Interest rate for closed contract.),num_group1,numberofcontrsvalue_258L (Number of active contracts in credit bureau.),numberofcontrsvalue_358L (Number of closed credit contracts.),numberofinstls_229L (Number of instalments on closed contract.),numberofinstls_320L (Number of instalments of the active contract.),numberofoutstandinstls_520L (Number of outstanding instalment for closed contract.),numberofoutstandinstls_59L (Number of outstanding instalments for the active contracts.),numberofoverdueinstlmax_1039L (Number of outstanding instalments for active contracts.),numberofoverdueinstlmax_1151L (Maximum number of past due installments for a closed contract.),numberofoverdueinstlmaxdat_148D (Date of maximum number of past due instalments for the closed contract.),numberofoverdueinstlmaxdat_641D (Date of maximum number of past due instalments for the active contract.),numberofoverdueinstls_725L (Maximum number of past due instalments for an active contract.),numberofoverdueinstls_834L (Number of past due instalments for a closed contract.),outstandingamount_354A (Outstanding amount for closed credit contract in credit bureau.),outstandingamount_362A (Active contract's outstanding amount.),overdueamount_31A (Past due amount for a closed contract.),overdueamount_659A (Past due amount for active contract.),overdueamountmax2_14A (Maximal past due amount for an active contract.),overdueamountmax2_398A (Maximal overdue amount for a closed contract.),overdueamountmax_155A (Maximal past due amount for active contract.),overdueamountmax_35A (Maximal past due amount for a closed contract.),periodicityofpmts_1102L (Frequency of instalments for a closed contract.),periodicityofpmts_837L (Frequency of instalments for an active contract.),prolongationcount_1120L (Count of prolongations on terminated contract according to credit bureau.),prolongationcount_599L (Count of active contract prolongations.),purposeofcred_426M (Purpose of credit for active contract.),purposeofcred_874M (Purpose of credit on a closed contract.),residualamount_488A (Residual amount of a closed contract.),residualamount_856A (Residual amount for the active contract.),subjectrole_182M (Subject role in active credit contract.),subjectrole_93M (Subject role in closed credit contract.),totalamount_6A (Total amount of closed contracts.),totalamount_996A (Total amount of active contracts in the credit bureau.),totaldebtoverduevalue_178A (Total amount of past due debt on active contracts.),totald